In [1]:
# Install required libraries
!pip install transformers datasets evaluate spacy rouge_score sacrebleu
!pip install datasets
!python -m spacy download en_core_web_sm
!pip install evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 626.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c07244e80be422ebcbe181ddd0394e6c07f017695cb2d25aa24146fe000db574
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 110.7 MB/s eta 0:00:00
✔ Download and i

In [4]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from evaluate import load
import spacy
from datasets import load_dataset

In [5]:
# Mount Google Drive
drive.mount("/content/drive")

# Paths to datasets
train_file_path = '/content/drive/MyDrive/Projet_LLM_UPVD/train.csv'
validation_file_path = '/content/drive/MyDrive/Projet_LLM_UPVD/validation.csv'
model_save_path = '/content/drive/MyDrive/fine_tuned_model'

# Load SpaCy model for preprocessing
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

# Load and preprocess datasets
def load_and_preprocess_dataset(train_path, val_path):
    train_dataset = load_dataset('csv', data_files={'train': train_path})['train']
    val_dataset = load_dataset('csv', data_files={'validation': val_path})['validation']
    train_dataset = train_dataset.map(lambda x: {"context_chunks": preprocess_text(x["context_chunks"])})
    val_dataset = val_dataset.map(lambda x: {"context_chunks": preprocess_text(x["context_chunks"])})
    return train_dataset, val_dataset

train_dataset, val_dataset = load_and_preprocess_dataset(train_file_path, validation_file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1348 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

In [6]:
# Load the lightweight model for Q&A
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Create QA pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Generate baseline output for text continuation
input_text = "COVID-19, caused by the SARS-CoV-2 virus, emerged in late 2019 in Wuhan, China, and quickly became a global pandemic. The virus is primarily transmitted through respiratory droplets..."
print("\nBaseline Text Continuation:")
generated_output = qa_pipeline(question="What is COVID-19?", context=input_text)["answer"]
print(f"Generated Text: {generated_output}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Device set to use cuda:0



Baseline Text Continuation:
Generated Text: caused by the SARS-CoV-2 virus


In [7]:

# Define questions and context for evaluation
questions = [
    "What is the main cause of HIV-1 infection in children?",
    "What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?",
    "What is DC-GENR and where is it expressed?"
]
context = """
HIV-1 infection in children is primarily caused by mother-to-child transmission. CCL3L1 competes for CCR5 binding,
reducing the risk of HIV-1 infection. DC-GENR is expressed on dendritic cells and plays a key role in HIV recognition.
"""

In [8]:

# Generate answers with the baseline model
def generate_baseline_answers(questions, context):
    results = {}
    for question in questions:
        answer = qa_pipeline(question=question, context=context)["answer"]
        results[question] = answer
    return results


In [9]:
# Few-Shot Evaluation for the specific questions
def few_shot_evaluation(questions, context, n_shots):
    results = {}
    for i in range(n_shots):
        question = questions[i]
        answer = qa_pipeline(question=question, context=context)["answer"]
        results[question] = answer
    return results



In [10]:
# Generate and display baseline answers
print("\nBaseline Model Answers:")
baseline_answers = generate_baseline_answers(questions, context)
for q, ans in baseline_answers.items():
    print(f"Q: {q}\nA: {ans}\n")



Baseline Model Answers:
Q: What is the main cause of HIV-1 infection in children?
A: mother-to-child transmission

Q: What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?
A: plays a key role in HIV recognition

Q: What is DC-GENR and where is it expressed?
A: on dendritic cells



In [11]:

# Few-Shot Evaluation
def few_shot_evaluation(dataset, n_shots):
    results = {}
    for i in range(n_shots):
        question = dataset[i]["question"]
        context = dataset[i]["context_chunks"]
        answer = qa_pipeline(question=question, context=context)["answer"]
        results[question] = answer
    return results

print("\nFew-Shot Answers:")
for n_shots in range(1, 6):  # From 1 to 5 shots
    few_shot_results = few_shot_evaluation(train_dataset, n_shots)
    print(f"\nFew-Shot Evaluation ({n_shots} shots):")
    for q, ans in few_shot_results.items():
        print(f"Q: {q}\nAnswer: {ans}\n")



Few-Shot Answers:

Few-Shot Evaluation (1 shots):
Q: What is the main cause of HIV-1 infection in children?
Answer: background mother child transmission MTCT


Few-Shot Evaluation (2 shots):
Q: What is the main cause of HIV-1 infection in children?
Answer: background mother child transmission MTCT

Q: What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk
Answer: DC SIGNR



You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Few-Shot Evaluation (3 shots):
Q: What is the main cause of HIV-1 infection in children?
Answer: background mother child transmission MTCT

Q: What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk
Answer: DC SIGNR

Q: How many children were infected by HIV-1 in 2008-2009, worldwide?
Answer: 90


Few-Shot Evaluation (4 shots):
Q: What is the main cause of HIV-1 infection in children?
Answer: background mother child transmission MTCT

Q: What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk
Answer: DC SIGNR

Q: How many children were infected by HIV-1 in 2008-2009, worldwide?
Answer: 90

Q: What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?
Answer: quantitative


Few-Shot Evaluation (5 shots):
Q: What is the main cause of HIV-1 infection in children?
Answer: background mother child transmission MTCT

Q: What plays the crucial role in the M

In [12]:
import torch
from torch.utils.data import DataLoader

# Preprocess function for tokenization
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["question"], examples["context_chunks"],
        truncation=True, padding="max_length", max_length=512
    )
    start_positions = []
    end_positions = []

    for i in range(len(examples["context_chunks"])):
        # Ensure `answer` key exists and fallback to empty string
        answer = examples.get("answer", [""])[i]
        start_idx = examples["context_chunks"][i].find(answer)

        if start_idx == -1:  # If answer is not found in the context
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_idx = start_idx + len(answer)
        tokenized_start = tokenizer(
            examples["context_chunks"][i], truncation=True, max_length=512
        ).char_to_token(start_idx)
        tokenized_end = tokenizer(
            examples["context_chunks"][i], truncation=True, max_length=512
        ).char_to_token(end_idx - 1)

        # Handle cases where char_to_token returns None
        start_positions.append(tokenized_start if tokenized_start is not None else 0)
        end_positions.append(tokenized_end if tokenized_end is not None else 0)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)

# Fine-Tuning Loop
model.train()
for epoch in range(2):  # Two epochs
    for batch in train_loader:
        inputs = tokenizer(
            batch["question"], batch["context_chunks"],
            truncation=True, padding="max_length", return_tensors="pt"
        ).to(model.device)

        # Validate batch size consistency
        if len(batch["start_positions"]) != inputs["input_ids"].shape[0]:
            print(f"Skipping batch due to mismatched label sizes. Inputs: {inputs['input_ids'].shape[0]}, Labels: {len(batch['start_positions'])}")
            continue

        labels = {
            "start_positions": torch.tensor(batch["start_positions"]).to(model.device),
            "end_positions": torch.tensor(batch["end_positions"]).to(model.device),
        }

        # Forward pass
        outputs = model(**inputs, start_positions=labels["start_positions"], end_positions=labels["end_positions"])
        loss = outputs.loss

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save the Fine-Tuned Model
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Fine-tuned model saved to {model_save_path}")


Map:   0%|          | 0/1348 [00:00<?, ? examples/s]

Map:   0%|          | 0/158 [00:00<?, ? examples/s]

<ipython-input-12-c7cbc7536b9d>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "start_positions": torch.tensor(batch["start_positions"]).to(model.device),
<ipython-input-12-c7cbc7536b9d>:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "end_positions": torch.tensor(batch["end_positions"]).to(model.device),


Fine-tuned model saved to /content/drive/MyDrive/fine_tuned_model


In [13]:
# Reload the fine-tuned model
model = AutoModelForQuestionAnswering.from_pretrained(model_save_path).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Define questions and context for QA
questions = [
    "What is the main cause of HIV-1 infection in children?",
    "What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?",
    "What is DC-GENR and where is it expressed?"
]
context = """
HIV-1 infection in children is primarily caused by mother-to-child transmission. CCL3L1 competes for CCR5 binding,
reducing the risk of HIV-1 infection. DC-GENR is expressed on dendritic cells and plays a key role in HIV recognition.
"""

# Generate answers
def generate_answers(questions, context):
    return {q: qa_pipeline(question=q, context=context)["answer"] for q in questions}

answers = generate_answers(questions, context)

# Display answers
print("\nFine-Tuned Model Answers:")
for q, a in answers.items():
    print(f"Q: {q}\nA: {a}\n")


Device set to use cuda:0



Fine-Tuned Model Answers:
Q: What is the main cause of HIV-1 infection in children?
A: mother-to-child transmission

Q: What is the role of C-C Motif Chemokine Ligand 3 Like 1 (CCL3L1) in mother to child transmission of HIV-1?
A: plays a key role in HIV recognition.

Q: What is DC-GENR and where is it expressed?
A: on dendritic cells



In [14]:
!pip install evaluate

In [15]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [16]:
import pandas as pd
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from evaluate import load
from sentence_transformers import SentenceTransformer, util

# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Define paths and models
validation_csv_path = '/content/drive/MyDrive/Projet_LLM_UPVD/validation.csv'  # Update with your dataset path
baseline_model_name = "distilbert-base-uncased-distilled-squad"  # Baseline model
fine_tuned_model_path = '/content/drive/MyDrive/fine_tuned_model'  # Fine-tuned model
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load data
df = pd.read_csv(validation_csv_path)
questions = df['question'].tolist()
contexts = [" ".join(eval(context)) for context in df['context_chunks']]  # Convert context_chunks into a single string
references = df['answer'].tolist()

# Validate loaded data
print("Questions:", questions[:5])
print("Contexts:", contexts[:5])
print("References:", references[:5])

# Initialize device and models
device = "cuda" if torch.cuda.is_available() else "cpu"
baseline_model = AutoModelForQuestionAnswering.from_pretrained(baseline_model_name).to(device)
fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(baseline_model_name)

# Set up QA pipelines
baseline_pipeline = pipeline("question-answering", model=baseline_model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)
fine_tuned_pipeline = pipeline("question-answering", model=fine_tuned_model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

# Generate predictions
baseline_predictions = [baseline_pipeline(question=q, context=c)['answer'] for q, c in zip(questions, contexts)]
fine_tuned_predictions = [fine_tuned_pipeline(question=q, context=c)['answer'] for q, c in zip(questions, contexts)]

# Load evaluation metric
bertscore_metric = load("bertscore")


# Format predictions and references
formatted_references = [" ".join(ref.split()) for ref in references]


# Compute BERTScore
bertscore_baseline = bertscore_metric.compute(predictions=baseline_predictions, references=references, lang="en")
bertscore_fine_tuned = bertscore_metric.compute(predictions=fine_tuned_predictions, references=references, lang="en")


# Display metrics comparison
print("\nMetrics Comparison:")
print(f"BERTScore (Baseline): {bertscore_baseline['f1']}")
print(f"BERTScore (Fine-Tuned): {bertscore_fine_tuned['f1']}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Metrics Comparison:
BERTScore (Baseline): [0.9999998807907104, 0.8609036207199097, 0.7759363055229187, 0.7720378637313843, 0.8374530076980591, 1.0000003576278687, 0.8094877600669861, 0.8619675636291504, 0.8474422693252563, 0.9060952067375183, 0.9766699075698853, 1.0000001192092896, 0.8277748227119446, 0.7695192098617554, 0.7930305600166321, 0.9431020021438599, 0.9192315936088562, 0.8202834725379944, 0.8642914891242981, 0.8948193788528442, 0.8735922574996948, 0.8306127786636353, 1.0000001192092896, 0.824085533618927, 0.946638822555542, 0.8072283864021301, 0.8358676433563232, 0.8117049932479858, 0.902328372001648, 0.8346841931343079, 1.0, 0.8390464782714844, 0.8494483828544617, 0.8696756958961487, 0.7882095575332642, 0.862820565700531, 0.8659210205078125, 0.9520716071128845, 0.8127949833869934, 0.8192076683044434, 0.8667005300521851, 0.8102971315383911, 0.766778826713562, 0.8596206903457642, 0.8084956407546997, 0.7722041010856628, 0.8362985253334045, 0.8099246025085449, 0.97288125753402